# 海龜交易策略回測範例

這個筆記本示範如何使用我們的交易策略回測框架來測試海龜交易策略。

In [ ]:
# 導入必要的套件
import os
import sys

sys.path.append('..')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from backtest_engine import BacktestEngine
from config.config import get_config
from strategies.turtle_strategy import TurtleStrategy
from utils import fetch_data, plot_price_and_signals, print_strategy_summary

## 1. 獲取市場資料

In [ ]:
# 選擇要測試的股票
symbol = 'NVDA'
period = '5y'  # 5年的資料

# 獲取資料
print(f"正在獲取 {symbol} 的資料...")
data = fetch_data(symbol, period=period)
print(f"獲取了 {len(data)} 筆資料")
data.head()

## 2. 創建交易策略

In [ ]:
# 創建海龜交易策略
strategy = TurtleStrategy(
    name="海龜策略 (20/10)",
    entry_window=20,
    exit_window=10,
    initial_capital=100000
)

print(f"策略名稱: {strategy.name}")
print(f"進場窗口: {strategy.entry_window} 天")
print(f"出場窗口: {strategy.exit_window} 天")
print(f"初始資金: ${strategy.initial_capital:,.2f}")

## 3. 運行回測

In [ ]:
# 創建回測引擎
backtest_config = get_config('backtest')
engine = BacktestEngine(
    initial_capital=backtest_config['initial_capital'],
    commission=backtest_config['commission']
)

# 運行回測
print("開始回測...")
result = engine.run_backtest(strategy, data, symbol)
print("回測完成!")

## 4. 分析結果

In [ ]:
# 顯示策略摘要
performance = result['performance']
print_strategy_summary(strategy.name, performance)

In [ ]:
# 顯示交易記錄並計算每次交易的損益
trades_df = pd.DataFrame(result['trades'])
if not trades_df.empty:
    print(f"總交易次數: {len(trades_df)}")
    
    # 計算每次交易的損益率
    trades_analysis = trades_df.copy()
    trades_analysis['trade_type'] = trades_analysis['quantity'].apply(lambda x: 'BUY' if x > 0 else 'SELL')
    
    # 配對買賣交易計算損益
    buy_trades = []
    sell_trades = []
    trade_pairs = []
    
    for i, trade in trades_analysis.iterrows():
        if trade['quantity'] > 0:  # 買入
            buy_trades.append({
                'index': i,
                'timestamp': trade['timestamp'],
                'quantity': abs(trade['quantity']),
                'price': trade['price'],
                'value': abs(trade['value']),
                'capital_after': trade['capital_after']
            })
        else:  # 賣出
            sell_trades.append({
                'index': i,
                'timestamp': trade['timestamp'],
                'quantity': abs(trade['quantity']),
                'price': trade['price'],
                'value': abs(trade['value']),
                'capital_after': trade['capital_after']
            })
    
    # 配對交易並計算損益
    for i, sell in enumerate(sell_trades):
        if i < len(buy_trades):
            buy = buy_trades[i]
            
            # 計算這次交易的損益
            buy_cost = buy['value']
            sell_proceeds = sell['value']
            profit_loss = sell_proceeds - buy_cost
            profit_loss_pct = (profit_loss / buy_cost) * 100
            
            # 計算持有天數
            holding_days = (sell['timestamp'] - buy['timestamp']).days
            
            trade_pairs.append({
                'trade_no': i + 1,
                'buy_date': buy['timestamp'].strftime('%Y-%m-%d'),
                'sell_date': sell['timestamp'].strftime('%Y-%m-%d'),
                'holding_days': holding_days,
                'quantity': buy['quantity'],
                'buy_price': buy['price'],
                'sell_price': sell['price'],
                'buy_cost': buy_cost,
                'sell_proceeds': sell_proceeds,
                'profit_loss': profit_loss,
                'profit_loss_pct': profit_loss_pct,
                'capital_after_buy': buy['capital_after'],
                'capital_after_sell': sell['capital_after'],
                'trade_result': 'WIN' if profit_loss > 0 else 'LOSS'
            })
    
    # 顯示交易配對結果
    if trade_pairs:
        trade_pairs_df = pd.DataFrame(trade_pairs)
        print(f"\n交易配對分析 (共 {len(trade_pairs)} 次完整交易):")
        print("=" * 140)
        
        for _, trade in trade_pairs_df.iterrows():
            print(f"交易 #{trade['trade_no']:2d}: {trade['buy_date']} -> {trade['sell_date']} "
                  f"({trade['holding_days']:3d}天) | "
                  f"買入: ${trade['buy_price']:7.2f} | 賣出: ${trade['sell_price']:7.2f} | "
                  f"損益: ${trade['profit_loss']:8.2f} ({trade['profit_loss_pct']:+6.2f}%) | "
                  f"資金: ${trade['capital_after_buy']:9.2f} -> ${trade['capital_after_sell']:9.2f} | "
                  f"{trade['trade_result']}")
        
        # 統計結果
        total_trades = len(trade_pairs_df)
        winning_trades = len(trade_pairs_df[trade_pairs_df['profit_loss'] > 0])
        losing_trades = len(trade_pairs_df[trade_pairs_df['profit_loss'] < 0])
        win_rate = (winning_trades / total_trades) * 100
        
        total_profit = trade_pairs_df['profit_loss'].sum()
        avg_profit = trade_pairs_df['profit_loss'].mean()
        avg_holding_days = trade_pairs_df['holding_days'].mean()
        
        avg_win = trade_pairs_df[trade_pairs_df['profit_loss'] > 0]['profit_loss'].mean() if winning_trades > 0 else 0
        avg_loss = trade_pairs_df[trade_pairs_df['profit_loss'] < 0]['profit_loss'].mean() if losing_trades > 0 else 0
        
        print(f"\n" + "=" * 60)
        print(f"交易統計摘要:")
        print(f"=" * 60)
        print(f"總交易次數: {total_trades}")
        print(f"獲利交易: {winning_trades} ({win_rate:.1f}%)")
        print(f"虧損交易: {losing_trades} ({100-win_rate:.1f}%)")
        print(f"總損益: ${total_profit:,.2f}")
        print(f"平均損益: ${avg_profit:,.2f}")
        print(f"平均持有天數: {avg_holding_days:.1f} 天")
        if winning_trades > 0:
            print(f"平均獲利: ${avg_win:,.2f}")
        if losing_trades > 0:
            print(f"平均虧損: ${avg_loss:,.2f}")
        if winning_trades > 0 and losing_trades > 0:
            profit_factor = abs(avg_win * winning_trades / (avg_loss * losing_trades))
            print(f"獲利因子: {profit_factor:.2f}")
        
        # 顯示最佳和最差交易
        best_trade = trade_pairs_df.loc[trade_pairs_df['profit_loss'].idxmax()]
        worst_trade = trade_pairs_df.loc[trade_pairs_df['profit_loss'].idxmin()]
        
        print(f"\n最佳交易: 交易#{best_trade['trade_no']} | ${best_trade['profit_loss']:,.2f} ({best_trade['profit_loss_pct']:+.2f}%)")
        print(f"最差交易: 交易#{worst_trade['trade_no']} | ${worst_trade['profit_loss']:,.2f} ({worst_trade['profit_loss_pct']:+.2f}%)")
        
        # 顯示資金變化
        print(f"\n資金變化摘要:")
        initial_capital = trade_pairs_df['capital_after_buy'].iloc[0] + trade_pairs_df['buy_cost'].iloc[0]
        final_capital = trade_pairs_df['capital_after_sell'].iloc[-1]
        print(f"初始資金: ${initial_capital:,.2f}")
        print(f"最終資金: ${final_capital:,.2f}")
        print(f"資金成長: ${final_capital - initial_capital:,.2f} ({((final_capital/initial_capital) - 1)*100:.2f}%)")
    
    print(f"\n原始交易記錄 (前10筆):")
    print(trades_df.head(10))
else:
    print("沒有執行任何交易")

## 5. 視覺化分析

In [ ]:
# 繪製價格和交易信號
signals_data = result['signals_data']
plot_price_and_signals(signals_data, f"{strategy.name} - {symbol}")

In [ ]:
# 繪製投資組合價值變化
portfolio_value = result['portfolio_value']
if portfolio_value:
    plt.figure(figsize=(12, 6))
    
    # 投資組合價值
    portfolio_series = pd.Series(portfolio_value, index=data.index[:len(portfolio_value)])
    plt.plot(portfolio_series.index, portfolio_series.values, label='投資組合價值', linewidth=2)
    
    # 買入持有策略比較
    buy_hold_value = (data['Close'] / data['Close'].iloc[0]) * strategy.initial_capital
    plt.plot(data.index, buy_hold_value, label='買入持有', linewidth=2, alpha=0.7)
    
    plt.title(f'{strategy.name} - 投資組合價值 vs 買入持有')
    plt.xlabel('日期')
    plt.ylabel('價值 ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # 計算買入持有的績效
    buy_hold_return = (buy_hold_value.iloc[-1] / strategy.initial_capital) - 1
    strategy_return = performance.get('total_return', 0)
    
    print(f"績效比較:")
    print(f"策略總回報: {strategy_return:.2%}")
    print(f"買入持有回報: {buy_hold_return:.2%}")
    print(f"超額回報: {(strategy_return - buy_hold_return):.2%}")